In [1]:
import requests

def download_landmarks_model():
    # URL for the dat file
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    
    # Download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open("shape_predictor_68_face_landmarks.dat.bz2", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    # The file is in .bz2 format, so we need to decompress it
    import bz2
    with bz2.open("shape_predictor_68_face_landmarks.dat.bz2", "rb") as f_src:
        with open("shape_predictor_68_face_landmarks.dat", "wb") as f_dest:
            f_dest.write(f_src.read())

# Call the function to download the model
download_landmarks_model()

In [3]:
import cv2
import dlib
import numpy as np

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load dlib's facial landmark predictor
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

# Open the webcam
cap = cv2.VideoCapture(0)

# Load the Sunglass image with Alpha channel
glassPNG = cv2.imread('sunglass.png',-1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        # Get the landmarks
        rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        landmarks = np.array([[p.x, p.y] for p in predictor(frame, rect).parts()])

        # Left Eye: landmarks[36:42]
        # Right Eye: landmarks[42:48]
        left_eye_center = np.mean(landmarks[36:42], axis=0).astype(int)
        right_eye_center = np.mean(landmarks[42:48], axis=0).astype(int)
        
        # Assuming you want the glasses to be between the two eyes
        scaling_factor = 1.5
        glasses_width = right_eye_center[0] - left_eye_center[0]
        glasses_height = int(glasses_width/3 * scaling_factor)

        overlay_glasses = cv2.resize(glassPNG, (glasses_width*2, int(glasses_width/1.5)))

        midpoint = ((left_eye_center[0] + right_eye_center[0]) // 2, (left_eye_center[1] + right_eye_center[1]) // 2)

        y1 = midpoint[1] - overlay_glasses.shape[0]//2
        y2 = y1 + overlay_glasses.shape[0]
        x1 = midpoint[0] - overlay_glasses.shape[1]//2
        x2 = x1 + overlay_glasses.shape[1]


        transparency_factor = 0.5
        alpha_s = (overlay_glasses[:, :, 3] / 255.0) * transparency_factor

        alpha_l = 1.0 - alpha_s
        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_s * overlay_glasses[:, :, c] + alpha_l * frame[y1:y2, x1:x2, c])


        for c in range(0, 3):
            frame[y1:y2, x1:x2, c] = (alpha_s * overlay_glasses[:, :, c] + alpha_l * frame[y1:y2, x1:x2, c])

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
